In [2]:
import pandas as pd
import tensorflow as tf
import numpy

In [45]:
model = tf.keras.models.load_model('modelos/NNModelBY_Best')

In [84]:
df = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'int64', 
    'geo_level_2_id':'int64',
    'geo_level_3_id':'int64', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

In [89]:
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id').apply(lambda x: x-1)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'int64', 
    'geo_level_2_id':'int64',
    'geo_level_3_id':'int64', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')


pd.options.display.float_format = '{:20,.2f}'.format

In [90]:
df = df_train_values_original[:173733]

In [91]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + str(column)
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [92]:
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df = one_hot_encode_data(df,'geo_level_1_id')
df.sort_values('geo_level_2_id')

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,geo_level_1_id_21,geo_level_1_id_22,geo_level_1_id_23,geo_level_1_id_24,geo_level_1_id_25,geo_level_1_id_26,geo_level_1_id_27,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_30
building_id,,,,,,,,,,,,,,,,,,,,,
440637,0,7341,2,30,15,5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
386440,0,1295,2,5,20,5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
390128,0,1728,1,10,5,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
456381,0,1295,2,10,21,6,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
533432,0,1295,2,10,15,5,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97398,1427,8766,2,15,10,6,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
294038,1427,222,2,20,11,6,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
631163,1427,6234,1,10,9,3,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [93]:
df = df.reset_index().merge(mean_df,on='geo_level_2_id',how='left').set_index('building_id')
df = df.drop(columns=['geo_level_2_id', 'geo_level_3_id'])
df

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,geo_level_1_id_24,geo_level_1_id_25,geo_level_1_id_26,geo_level_1_id_27,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_30,0,1,2
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0.01,0.27,0.72
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.01,0.50,0.49
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.08,0.31,0.61
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.04,0.84,0.12
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.04,0.55,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823257,3,15,10,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.04,0.38,0.58
373540,2,15,7,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.01,0.50,0.49
627590,4,80,8,9,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.04,0.71,0.25


In [94]:
mean_df = pd.read_csv('geolevel_2_id_mean.csv',low_memory=False)
mean_df

,geo_level_2_id,0,1,2
0,0,0.04,0.40,0.56
1,1,0.01,0.61,0.38
2,3,0.04,0.73,0.23
3,4,0.01,0.43,0.56
4,5,0.04,0.77,0.19
...,...,...,...,...
1393,1423,0.10,0.57,0.33
1394,1424,0.07,0.69,0.24
1395,1425,0.00,0.31,0.69
1396,1426,0.01,0.21,0.78


In [95]:
average_damage = pd.get_dummies(df_train_labels_original['damage_grade'][:int(len(df_train_labels_original)/2)]).mean().values
average_damage

array([0.09630852, 0.56933998, 0.3343515 ])

In [96]:
df['0'] = df['0'].fillna(average_damage[0])
df['1'] = df['1'].fillna(average_damage[1])
df['2'] = df['2'].fillna(average_damage[2])

In [ ]:
test_database = df.to_numpy()
predictions = model(test_database).numpy()
pd.DataFrame(predictions).set_index(df.index).to_csv("ensamble_train_feda.csv")

Int64Index([802906,  28830,  94947, 590882, 201944, 333020, 728451, 475515,
            441126, 989500,
            ...
            142017, 651818, 584379, 802531, 544902, 823257, 373540, 627590,
            421951, 241191],
           dtype='int64', name='building_id', length=173733)

In [74]:
test_database = df.to_numpy()
predictions = model(test_database).numpy()
building_ids = df.reset_index()['building_id']
results = []
for line in predictions:
    if line[0] > line [1] and line [0] > line [2]:
        results.append(1)
    elif line[1] > line[2]:
        results.append(2)
    else:
        results.append(3)
result_series = pd.Series(results).rename("damage_grade")
my_submission = pd.merge(building_ids,result_series,left_index=True,right_index=True).set_index('building_id')

In [75]:
my_submission.to_csv('submission_feda_6.csv')